<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Google-Cloud-Speech-to-Text-API-Long_Running-example" data-toc-modified-id="Google-Cloud-Speech-to-Text-API-Long_Running-example-1">Google Cloud Speech-to-Text API Long_Running example</a></span></li></ul></div>

# Google Cloud Speech-to-Text API Long_Running example

- 긴 오디오 파일을 비동기 방식으로 처리하는 샘플
- **오디오 파일은 Google Cloud Storage에 업로드 후 사용**
- **오디오 파일 크기 제한은 없으나, 전체 파일을 모두 변환 후 응답을 받을 수 있음**
- https://cloud.google.com/speech-to-text/docs/async-recognize
- https://cloud.google.com/speech-to-text/docs/async-time-offsets
- Google Cloud Storage 설정 방법 : https://cloud.google.com/storage/docs/quickstart-console

In [1]:
"""Transcribe the given audio file asynchronously and output the word time offsets."""
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

client = speech.SpeechClient()

# gcs_uri = "gs://aiqa-stt/경제청문회_4명인터뷰_YTN_16000.wav"    # jupyter notebook에서 Cloud Storage 한글 파일명 인식 오류
gcs_uri = "gs://aiqa-stt/economy_4_16000.wav"

audio = types.RecognitionAudio(uri=gcs_uri)
config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='ko-KR',
    enable_automatic_punctuation=True,
    enable_word_time_offsets=True)

operation = client.long_running_recognize(config, audio)

In [2]:
print('Waiting for operation to complete...')
response = operation.result(timeout=360)
print('operation complete !!!')

Waiting for operation to complete...
operation complete !!!


In [3]:
type(response)

google.cloud.speech_v1.types.LongRunningRecognizeResponse

In [4]:
response.results[0]

alternatives {
  transcript: "\355\225\234\352\265\255 \353\225\205\354\235\264 \353\202\264 \352\261\264 \352\262\275\354\240\234\354\262\255\353\254\270\355\232\214 \354\227\254\354\225\274 \352\260\201 \354\240\225\353\213\271 \354\231\234 \353\260\230\354\235\221\354\235\264 \354\227\206\354\226\264 \354\225\214\353\246\275\353\213\210\353\213\244. \353\257\274\354\243\274\353\213\271\352\263\274 \353\260\224\353\245\270\353\257\270\353\236\230\353\213\271 \354\240\225\354\235\230\353\213\271\354\235\200 \355\225\234\352\265\255\353\213\271\354\235\264 \354\226\265\354\247\200\353\245\274 \353\266\200\353\246\254\353\212\224 \355\225\230\353\212\224 \354\236\205\354\236\245\354\235\270 \353\260\230\353\251\264\354\227\220 \355\225\234\352\265\255\353\213\271\352\263\274 \353\257\274\354\243\274\355\217\211\355\231\224\353\213\271\354\235\200 \353\257\274\354\243\274\353\213\271\354\235\264 \354\262\255\353\254\270\355\232\214\353\245\274 \353\260\233\354\225\204\354\225\274 \355\22

In [5]:
response.results[0].alternatives[0].transcript

'한국 땅이 내 건 경제청문회 여야 각 정당 왜 반응이 없어 알립니다. 민주당과 바른미래당 정의당은 한국당이 억지를 부리는 하는 입장인 반면에 한국당과 민주평화당은 민주당이 청문회를 받아야 한다고 압박합니다.'

In [6]:
response.results[0].alternatives[0].confidence

0.9035612940788269

In [7]:
print("start_time:", response.results[0].alternatives[0].words[0].start_time.seconds + \
      response.results[0].alternatives[0].words[0].start_time.nanos * 1e-9)

start_time: 0.30000000000000004


In [8]:
print("end_time:", response.results[0].alternatives[0].words[-1].end_time.seconds + \
     response.results[0].alternatives[0].words[-1].end_time.seconds * 1e-9)

end_time: 15.000000015


In [9]:
for result in response.results:
    alternative = result.alternatives[0]
    print(u'Transcript: {}'.format(alternative.transcript))
    print('Confidence: {}'.format(alternative.confidence))

    start_time = alternative.words[0].start_time
    end_time = alternative.words[-1].end_time
    print('start_time: {} ~ end_time: {}'.format(start_time.seconds + start_time.nanos * 1e-9, \
            end_time.seconds + end_time.nanos * 1e-9))
    print()

Transcript: 한국 땅이 내 건 경제청문회 여야 각 정당 왜 반응이 없어 알립니다. 민주당과 바른미래당 정의당은 한국당이 억지를 부리는 하는 입장인 반면에 한국당과 민주평화당은 민주당이 청문회를 받아야 한다고 압박합니다.
Confidence: 0.9035612940788269
start_time: 0.30000000000000004 ~ end_time: 15.2

Transcript:  여론은 반달과 55%로 찬성 31%를 크게 앞지른 것으로 조사됐습니다. 반대 여론이 높은 경제청문회를 내 건 한국 땅이 문제가 앞으로 정당지지도 그러면 어떤 영향을 미칠지 궁금합니다. 숫자에 동지 더 여론에서 YTN 더뉴스 가욋일 시안 여론조사 결과 분석하겠습니다. 이택수 리얼미터대표 박시영 윈지코리아 컨설팅 부대표 타기 하셨습니다. 안녕하십니까 안녕하세요 안녕하세요 경제청문회 관련한 조사 결과부터 다시 한번 정리를 하고 얘기를 이어가겠습니다. 함께보시죠. 한국 땅이 요구한 경제청문회 응답자의 55.4%가 반대하는 것으로 나타났습니다. 찬성은 30.9% 였고요. 모른다거나 응답하지않음. 비율은 10.7%였습니다. 응답자를 지지정당 별로 나눠서 살펴 보겠습니다. 민주당 10층에 결석도 같아서 높았습니다. 응답자중 민주당 지지층에 85점.
Confidence: 0.9158132672309875
start_time: 16.7 ~ end_time: 76.6

Transcript:  퍼센트는 반대했고, 한국당을 지지하는 응답자는 68.2%가 찬성하는 것으로 나타났습니다. 정치 성향 별로 나눠서 조사결과에서도 4층에 반대여론이 보수층의 찬성여론 보다 경고했습니다. 보수측에서의 반대 여론이 35.8%로 경제청문회를 정치공세로 보는 시각도 포스팅에서도 적지않아 중독증은 전체 여론 부도와 비슷했습니다. 정치 성향 결론은 중도층이 전체 여론과 비슷해 따면 지역별로 나는 수도권과 충청권을 그랬습니다. 결과를 좀 봤습니다. 찬성 30.9% 돼 반대 55.4% 대표님 한국 땅에 이런 여러 오늘

In [10]:
# CSV 파일로 저장
import csv

with open('text/경제청문회_4명인터뷰_YTN_16000.csv', 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["start_time", "end_time", "transcript"])
    for result in response.results:
        alternative = result.alternatives[0]
        start_time = alternative.words[0].start_time.seconds + alternative.words[0].start_time.nanos * 1e-9
        end_time = alternative.words[-1].end_time.seconds + alternative.words[-1].end_time.nanos * 1e-9
        wr.writerow([start_time, end_time, alternative.transcript])

In [11]:
# 스크립트 만 Text 파일로 저장
with open('text/경제청문회_4명인터뷰_YTN_16000.txt', 'w', encoding='utf-8', newline='') as f:
    for result in response.results:
        alternative = result.alternatives[0]
        f.write(alternative.transcript + " ")